In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import re
import os

In [25]:
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.preprocessing import normalize
from tqdm import tqdm, tqdm_pandas

import nltk
from nltk.corpus import stopwords
from sklearn.metrics import accuracy_score, precision_score, recall_score
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer
nltk.download('wordnet')

import re

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/alanliu99/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
#os.chdir('../')
business = pd.read_csv('data/business.csv')
#review = pd.read_csv('test/testdata/test_review.csv')
las_vagas_reviews = pd.read_csv('data/Las_Vegas_review.csv')

In [4]:
reviews = pd.read_csv('data/reviews.csv')

In [15]:
las_vagas_reviews.text= las_vagas_reviews.text.fillna("")

In [16]:
rest = las_vagas_reviews[las_vagas_reviews.text.apply(lambda x: len(x.split())) > 50]
rest

,name,business_id,city,categories,review_id,text,stars,user_id
0,Carl's Jr,fnZrZlqW1Z8iWgTVDfv_MA,Las Vegas,"Mexican, Restaurants, Fast Food",WUAt5-krh075Ie44np8Hew,So why would I be giving a Fast Food chain loc...,5.0,YDkC5VVT8s9NlIZtAr8NUA
1,Carl's Jr,fnZrZlqW1Z8iWgTVDfv_MA,Las Vegas,"Mexican, Restaurants, Fast Food",S9vIs-je49jqbhpWSQ8yHg,I come here bout 3x's a mo. & I just can't do ...,2.0,mxtalrN7VHVQwGjv8ln5yg
2,Carl's Jr,fnZrZlqW1Z8iWgTVDfv_MA,Las Vegas,"Mexican, Restaurants, Fast Food",i2Soid_X8XsSTyIrPOst6g,WORST experience EVER!!!!! never have i ate an...,1.0,xammeA3ftpFoAN13v5Blyg
3,Carl's Jr,fnZrZlqW1Z8iWgTVDfv_MA,Las Vegas,"Mexican, Restaurants, Fast Food",uDYGaAOUgo7SLuH5nUZs3w,Hot fresh food usually. Staff seems to turn o...,4.0,gQhFacMGI41mfVXqw1E7bQ
7,Carl's Jr,fnZrZlqW1Z8iWgTVDfv_MA,Las Vegas,"Mexican, Restaurants, Fast Food",eDqCFNQyAv74NID-BcT5KA,"Food was alright, but with only 1 person ahead...",2.0,CMMlQQxJXr6f4Jop_ozZKA
...,...,...,...,...,...,...,...,...
1503838,Island Buffet,wjZ12shpUWAy4Boy0G5z1g,Las Vegas,"Restaurants, Buffets",reGRBX3twj8OJnYbtje1SA,"Stay away, far, far, far away from here! We so...",1.0,iCos25g_O3Z5Am9K4383WA
1503840,Island Buffet,wjZ12shpUWAy4Boy0G5z1g,Las Vegas,"Restaurants, Buffets",IeowybrKAmmf9njf7pa1rA,So there are alot of pretty bad reviews but I ...,3.0,F5YcdVMqhDAfhGZsMCXDTA
1503841,Island Buffet,wjZ12shpUWAy4Boy0G5z1g,Las Vegas,"Restaurants, Buffets",r7maiBTmlfFRcaLAmwY0Aw,All the buffets are pretty much the same. Ove...,2.0,CPLndzPfGfJzjqjzXCr9_g
1503842,Island Buffet,wjZ12shpUWAy4Boy0G5z1g,Las Vegas,"Restaurants, Buffets",UPn6fAXxo7SEaQL4xkUgFA,Don't come here for dinner. You can see the b...,1.0,pSoe4XBftnJiTgG1fPw52g


In [18]:
test_1000_id = np.unique(rest.business_id)[0:1000]
test_df = rest[rest.business_id.isin(test_1000_id)].copy()

In [26]:
%%time
#Lemmatize, remove punctuations and symbols
lmtzr = WordNetLemmatizer()
tokenizer = TweetTokenizer()
sent_pre1 = test_df['text'].apply(lambda a: [lmtzr.lemmatize(word) for word in tokenizer.tokenize(a.lower())]) 

CPU times: user 3min 6s, sys: 3.43 s, total: 3min 9s
Wall time: 3min 24s


In [27]:
sent_pre1

4786       [this, little, hidden, gem, of, dtlv, is, my, ...
4787       [i, wish, there, were, more, place, like, ogde...
4788       [my, favorite, place, for, sandwich, &, the, o...
4789       [i, really, appreciate, that, there, is, a, co...
4790       [this, place, is, the, best, ., i, work, right...
                                 ...                        
1503829    [i'm, a, bit, annoyed, by, the, yelp, rating, ...
1503831    [wow, ,, if, i, could, give, this, place, no, ...
1503832    [i, love, noodle, and, saw, this, restaurant, ...
1503833    [one, thing, i, noticed, while, walking, in, /...
1503834    [i've, only, had, the, sushi, ..., but, it, wa...
Name: text, Length: 144207, dtype: object

In [28]:
sent_pre2 = sent_pre1.apply(lambda a:[word for word in a if ((re.search(r'[^\w\s]', word) is None)|(len(word)>1))])

In [ ]:
st_W = set([lmtzr.lemmatize(word) for word in stopwords.words('english')])
tokenised_sentences =  sent_pre2  